# Customer Analysis - Explore Customer Behavior

## Import

Needed packages. Pyspark will be used for data managing and plotly for visualisations. Keep in mind to install
JAVA so Spark will work properly.

Used dataset is from https://rees46.com/de found on https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store.

In [34]:
import os
import pyspark
import pandas as pd
import pyspark.sql.functions as f
import plotly.express as px
import plotly.graph_objects as go

## Read

The data needs to be located in ```data/``` in unzipped form as a csv.

In [35]:
# read raw data
spark = pyspark.sql.SparkSession.builder.appName("app1").getOrCreate()
# sdf = spark.read.csv("data/*.csv", header=True, inferSchema=True)
sdf_201911 = spark.read.csv("data/2019-Nov.csv", header=True, inferSchema=True)
sdf_201910 = spark.read.csv("data/2019-Oct.csv", header=True, inferSchema=True)

In [36]:
# join both months together
sdf = sdf_201910.union(sdf_201911)
# sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)
sdf.show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|elect

## Preparation

Prepare and enhance data for analysis and modelling.

In [37]:
# Datatypes
sdf = sdf.withColumn("event_time", sdf["event_time"].cast(pyspark.sql.types.TimestampType()))
sdf = sdf.withColumn("category_id", sdf["category_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("product_id", sdf["product_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("user_id", sdf["user_id"].cast(pyspark.sql.types.StringType()))

# Feature Splitting
# sdf = sdf.withColumn("category_class", f.substring_index(sdf.category_code, '.', 1))

sdf = sdf.withColumn("category_class", f.split(sdf["category_code"], "\.").getItem(0))
sdf = sdf.withColumn("category_sub_class", f.split(sdf["category_code"], "\.").getItem(1))
sdf = sdf.withColumn("category_sub_sub_class", f.split(sdf["category_code"], "\.").getItem(2))

sdf = sdf.withColumn("year", f.year("event_time"))
sdf = sdf.withColumn("month", f.month("event_time"))
sdf = sdf.withColumn("weekofyear", f.weekofyear("event_time"))
sdf = sdf.withColumn("dayofyear", f.dayofyear("event_time"))
sdf = sdf.withColumn("dayofweek", f.dayofweek("event_time"))
sdf = sdf.withColumn("dayofmonth", f.dayofmonth("event_time"))
sdf = sdf.withColumn("hour", f.hour("event_time"))
# None Handling
# sdf = sdf.fillna(value="not defined")

sdf.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)
 |-- category_class: string (nullable = true)
 |-- category_sub_class: string (nullable = true)
 |-- category_sub_sub_class: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- dayofyear: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- hour: integer (nullable = true)



## Dataframe Creation

create several dataframes with different aggregation level to answer different questions/ tasks.

In [38]:
# raw
sdf_raw = sdf
sdf_raw.show(1, vertical=True)

-RECORD 0--------------------------------------
 event_time             | 2019-11-01 01:00:00  
 event_type             | view                 
 product_id             | 1003461              
 category_id            | 2053013555631882655  
 category_code          | electronics.smart... 
 brand                  | xiaomi               
 price                  | 489.07               
 user_id                | 520088904            
 user_session           | 4d3b30da-a5e4-49d... 
 category_class         | electronics          
 category_sub_class     | smartphone           
 category_sub_sub_class | null                 
 year                   | 2019                 
 month                  | 11                   
 weekofyear             | 44                   
 dayofyear              | 305                  
 dayofweek              | 6                    
 dayofmonth             | 1                    
 hour                   | 1                    
only showing top 1 row



In [39]:
# aggregated customer
sdf_agg_cust = sdf.groupBy("user_id", "user_session", "event_type", "product_id").mean("price")
sdf_agg_cust.show(5)

+---------+--------------------+----------+----------+----------+
|  user_id|        user_session|event_type|product_id|avg(price)|
+---------+--------------------+----------+----------+----------+
|513998949|a7b196d9-afe5-4dc...|      view|  50600085|    113.93|
|565731881|5d8cb7aa-ca44-470...|      view|   4804194|     69.24|
|565731881|5d8cb7aa-ca44-470...|      view|   4804151|     51.22|
|550043341|200ebe4a-40e6-4c3...|      view|  16000004|     43.73|
|532647354|d2d3d2c6-631d-489...|      view|   1004258|    732.07|
+---------+--------------------+----------+----------+----------+
only showing top 5 rows



In [40]:
# aggregated session

In [41]:
# aggregated product


In [42]:
# aggregated class

In [43]:
# aggregated time (weeks, dayofweeks, month)

sdf_time_dist_month = sdf.groupBy("event_type", "dayofmonth").count()
sdf_time_dist_month = sdf_time_dist_month.withColumnRenamed("count", "cnt")
sdf_time_dist_month = sdf_time_dist_month.sort("dayofmonth", "event_type")
sdf_time_dist_month.show()

+----------+----------+---+
|event_type|dayofmonth|cnt|
+----------+----------+---+
|      cart|         1|  1|
|  purchase|         1|  2|
|      view|         1|214|
|      view|         9|  8|
|      cart|        12|  1|
|      view|        12| 18|
+----------+----------+---+



In [44]:
sdf_time_dist_week = sdf.groupBy("event_type", "dayofweek").count()
sdf_time_dist_week = sdf_time_dist_week.withColumnRenamed("count", "cnt")
sdf_time_dist_week = sdf_time_dist_week.sort("dayofweek", "event_type")
sdf_time_dist_week.show()

+----------+---------+---+
|event_type|dayofweek|cnt|
+----------+---------+---+
|      cart|        3|  1|
|      view|        3| 18|
|      view|        4|  8|
|      cart|        6|  1|
|  purchase|        6|  2|
|      view|        6|214|
+----------+---------+---+



In [45]:
sdf_time_dist_day = sdf.groupBy("event_type", "hour").count()
sdf_time_dist_day = sdf_time_dist_day.withColumnRenamed("count", "cnt")
sdf_time_dist_day = sdf_time_dist_day.sort("hour", "event_type")
sdf_time_dist_day.show()

+----------+----+---+
|event_type|hour|cnt|
+----------+----+---+
|      cart|   1|  1|
|  purchase|   1|  2|
|      view|   1|214|
|      cart|   2|  1|
|      view|   2| 18|
|      view|  16|  8|
+----------+----+---+



## Field Explanations

Following fields are in the standard dataset:
- event_time
- product_id
- category_id
- category_code
- brand
- price
- user_id
- user_session

### general

In [46]:
sdf_raw.show(5)
print(f"Number of total rows: {sdf_raw.count()}")

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+--------------+------------------+----------------------+----+-----+----------+---------+---------+----------+----+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|category_class|category_sub_class|category_sub_sub_class|year|month|weekofyear|dayofyear|dayofweek|dayofmonth|hour|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+--------------+------------------+----------------------+----+-----+----------+---------+---------+----------+----+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|   electronics|        smartphone|                  null|2019|   11|        44|      305|        6|         1|   1|
|2019-11-01 01:00:00

### event_time

In [47]:
sdf_raw.select("event_time").show(5)
print(f"Number of distinct event_time rows: {sdf_raw.select('event_time').distinct().count()}")

+-------------------+
|         event_time|
+-------------------+
|2019-11-01 01:00:00|
|2019-11-01 01:00:00|
|2019-11-01 01:00:01|
|2019-11-01 01:00:01|
|2019-11-01 01:00:01|
+-------------------+
only showing top 5 rows

Number of distinct event_time rows: 84


### event_type

In [48]:
print("Unique Values:")
sdf_raw.select("event_type").distinct().show()

print("Distribution:")
sdf_event_type_dist = sdf_raw.groupBy("event_type").count()
sdf_event_type_dist.show()

Unique Values:
+----------+
|event_type|
+----------+
|  purchase|
|      view|
|      cart|
+----------+

Distribution:
+----------+-----+
|event_type|count|
+----------+-----+
|  purchase|    2|
|      view|  240|
|      cart|    2|
+----------+-----+



In [49]:
# Plot Event Types
df = sdf_event_type_dist.toPandas()
fig = px.pie(df, values='count', names='event_type', title='Distribution of Customer Actions')
fig.show()

### product_id

In [50]:
sdf_raw.select("product_id").show(5)

+----------+
|product_id|
+----------+
|   1003461|
|   5000088|
|  17302664|
|   3601530|
|   1004775|
+----------+
only showing top 5 rows



In [51]:
print("Number of unique values:")
print(sdf_raw.select("product_id").distinct().count())

Number of unique values:
214


In [52]:
print("Number of products with no category_id:")
# print(sdf_raw.where((sdf_raw["category_id"] == "not defined")).count())
# print(sdf_raw.na.count())

Number of products with no category_id:


In [53]:
print("Distribution:")
sdf_product_id_dist = sdf_raw.groupBy("product_id").count()
sdf_product_id_dist.show()

Distribution:
+----------+-----+
|product_id|count|
+----------+-----+
|   1306571|    1|
|  25600085|    1|
|  26402378|    1|
|   3901174|    1|
|   5801656|    1|
|   1307012|    1|
|  26204072|    1|
|   3100034|    1|
|   2300214|    1|
|  15100148|    1|
|   4804194|    1|
|   3701016|    1|
|  12708306|    1|
|  16700690|    1|
|   1307322|    1|
|   2400216|    1|
|  41100055|    1|
|   1307115|    2|
|   4600541|    1|
|  15100337|    1|
+----------+-----+
only showing top 20 rows



In [54]:
px.bar(sdf_product_id_dist.orderBy("count").limit(10).toPandas(), x='product_id', y='count')

### category_id

In [55]:
sdf_raw.select("category_id").show(5)
print(f"Number of unique categories: {sdf_raw.select('category_id').distinct().count()}")

+-------------------+
|        category_id|
+-------------------+
|2053013555631882655|
|2053013566100866035|
|2053013553853497655|
|2053013563810775923|
|2053013555631882655|
+-------------------+
only showing top 5 rows

Number of unique categories: 90


In [56]:
sdf_product_per_category = sdf_raw.groupBy("category_id").agg(f.countDistinct("product_id"))
sdf_product_per_category.show()

+-------------------+-----------------+
|        category_id|count(product_id)|
+-------------------+-----------------+
|2085718636156158307|                1|
|2152167773222993940|                1|
|2053013558190408249|                1|
|2146660887346282824|                1|
|2053013558031024687|                1|
|2090971680431145002|                1|
|2053013558282682943|                1|
|2053013556110033341|                1|
|2053013558920217191|               17|
|2134905044833666047|                2|
|2053013560530830019|                1|
|2106075662325383725|                1|
|2070747671722722162|                1|
|2137704922018218396|                2|
|2053013559792632471|                2|
|2053013554625249641|                1|
|2053013562183385881|                1|
|2053013554449088861|                1|
|2053013558433677895|                1|
|2134905045613805589|                1|
+-------------------+-----------------+
only showing top 20 rows



In [57]:
df = sdf_product_per_category.toPandas()
px.box(df, y="count(product_id)")

### category_code

In [58]:
sdf_raw.select("category_code").show(5)

print("Number of unique categorie codes:")
print(sdf_raw.select("category_code").distinct().count())

+--------------------+
|       category_code|
+--------------------+
|electronics.smart...|
|appliances.sewing...|
|                null|
|appliances.kitche...|
|electronics.smart...|
+--------------------+
only showing top 5 rows

Number of unique categorie codes:
42


### brand

In [59]:
sdf_raw.select("brand").show(5)

print("Number of unique brands:")
print(sdf_raw.select("brand").distinct().count())

+------+
| brand|
+------+
|xiaomi|
|janome|
| creed|
|    lg|
|xiaomi|
+------+
only showing top 5 rows

Number of unique brands:
99


### price

In [60]:
sdf_raw.describe("price").show()

+-------+-----------------+
|summary|            price|
+-------+-----------------+
|  count|              244|
|   mean|302.1450819672131|
| stddev|425.3893301363122|
|    min|             1.09|
|    max|          2496.59|
+-------+-----------------+



In [61]:
df = sdf_raw.toPandas()
px.box(df, y="price")


### user_id

In [62]:
sdf_raw.select("user_id").show(5)

print("Number of users:")
print(sdf_raw.select("user_id").distinct().count())

+---------+
|  user_id|
+---------+
|520088904|
|530496790|
|561587266|
|518085591|
|558856683|
+---------+
only showing top 5 rows

Number of users:
153


### user_session

In [63]:
# avg actions per session
sdf_cnt_action_per_session = sdf_raw.groupby("user_session").count()
sdf_cnt_action_per_session.describe().show()

+-------+--------------------+------------------+
|summary|        user_session|             count|
+-------+--------------------+------------------+
|  count|                 153|               153|
|   mean|                null|1.5947712418300655|
| stddev|                null|1.0287825488662496|
|    min|0110890b-96d6-4ae...|                 1|
|    max|ff868137-fb3e-4da...|                 6|
+-------+--------------------+------------------+



## Exploration and Analysis

### Time Distribution

In [64]:
# Timestamp Distribution (per event_type) over every day of month

df = sdf_time_dist_month.toPandas()

fig = px.bar(df, x = 'dayofmonth', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a month",
     xaxis_title = 'Day of Month', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

In [65]:
# Timestamp Distribution (per event_type) over every day of week

df = sdf_time_dist_week.toPandas()

fig = px.bar(df, x = 'dayofweek', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a week",
     xaxis_title = 'Day of Week', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

In [67]:
# Timestamp Distribution (per event_type) over every hour of a day

df = sdf_time_dist_day.toPandas()

fig = px.bar(df, x = 'hour', y = 'cnt', color ='event_type', barmode = 'stack')

fig.update_layout(title = "Number of events over a day",
     xaxis_title = 'Hour of day', yaxis_title = 'Number of Events')
fig.update_xaxes(type="category")
fig.show()

In [71]:
sdf.groupBy("event_time").count().show()

+-------------------+-----+
|         event_time|count|
+-------------------+-----+
|2019-11-01 01:00:09|    3|
|2019-11-01 01:01:23|    3|
|2019-11-01 01:00:19|    5|
|2019-11-01 01:00:41|    7|
|2019-11-01 01:01:00|    2|
|2019-11-01 01:01:17|    2|
|2019-11-01 01:00:35|    3|
|2019-11-01 01:01:20|    3|
|2019-11-01 01:01:21|    3|
|2019-11-01 01:00:27|    5|
|2019-11-01 01:01:11|    4|
|2019-11-01 01:01:07|    1|
|2019-11-01 01:00:12|    5|
|2019-11-01 01:00:53|    2|
|2019-11-01 01:00:22|    2|
|2019-11-01 01:00:54|    1|
|2019-11-01 01:00:16|    4|
|2019-11-01 01:00:51|    3|
|2019-11-01 01:01:01|    3|
|2019-11-01 01:00:26|    4|
+-------------------+-----+
only showing top 20 rows



In [73]:
df = sdf.groupBy("event_time").count().toPandas()

fig = px.line(df, x="event_time", y="count")
fig.show()

In [68]:
df = px.data.stocks()
df.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


### Category and products

### Price and Events

### User analysis

### Customer Profiles

### Customer Journey

## Clustering